## Решающие деревья

В библиотеке **scikit-learn** решающие деревья реализованы в классах 

1. sklearn.tree.DecisionTreeСlassifier (для классификации)
2. sklearn.tree.DecisionTreeRegressor (для регрессии).

Обучение модели производится с помощью функции fit.

In [186]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# import seaborn as sns

### 1. Загрузка данных и первичный просмотр

In [187]:
# Загрузите выборку из файла titanic.csv с помощью пакета Pandas.

data = pd.read_csv('titanic.csv', index_col = 'PassengerId')

In [188]:
data.head(n=3)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [189]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [190]:
data.shape

(891, 11)

In [191]:
# посмотрим, сколько пропущеных значений в df

data.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [192]:
# посмотрим на количество уникальных значений в каждом столбце
# для последующего определения категориальных признаков

data.nunique()

Survived      2
Pclass        3
Name        891
Sex           2
Age          88
SibSp         7
Parch         7
Ticket      681
Fare        248
Cabin       147
Embarked      3
dtype: int64

### 2. Подготовка таблицы "обьект-признак" для алгоритма обучения

In [193]:
# Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
# с помощью метода loc считаем фрагмент данных

table = data.loc[:,['Pclass','Fare','Age','Sex']]

In [194]:
table.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,male
2,1,71.2833,38.0,female
3,3,7.9250,26.0,female
4,1,53.1000,35.0,female
5,3,8.0500,35.0,male


### 3. Кодирование категориальных признаков

In [195]:
# Обратите внимание, что признак Sex имеет строковые значения.
# преобразуем нечисловые данные в числовые
# LabelEncoder будет заменять «мужские» значения одним числом (1), а «женские» значения — другим числом (0).

table['Sex'] = preprocessing.LabelEncoder().fit_transform(table['Sex'])

In [196]:
table.head()

,Pclass,Fare,Age,Sex
PassengerId,,,,
1,3,7.2500,22.0,1
2,1,71.2833,38.0,0
3,3,7.9250,26.0,0
4,1,53.1000,35.0,0
5,3,8.0500,35.0,1


### 4. Работа с пропущенными данными

In [197]:
# В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст.
# Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

# посмотрим, сколько в нашейт тавлице пропущенных данных

table.isnull().sum()

Pclass      0
Fare        0
Age       177
Sex         0
dtype: int64

In [198]:
# удалим строки с пропущенными значениями

table = table.dropna()

In [199]:
table.isnull().sum()

Pclass    0
Fare      0
Age       0
Sex       0
dtype: int64

In [200]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 1 to 891
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  714 non-null    int64  
 1   Fare    714 non-null    float64
 2   Age     714 non-null    float64
 3   Sex     714 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 27.9 KB


### 4. Выделение целевой переменной

In [201]:
# Выделите целевую переменную — она записана в столбце Survived.

ind_val = data['Survived']

In [202]:
ind_val

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

In [203]:
ind_val = ind_val[table.index.values]

In [204]:
ind_val

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
886    0
887    0
888    1
890    1
891    0
Name: Survived, Length: 714, dtype: int64

### 5. Обучение

In [205]:
# Обучите решающее дерево с параметром random_state=241
# и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

model = DecisionTreeClassifier(random_state=241)

In [206]:
model.fit(table,ind_val)

DecisionTreeClassifier(random_state=241)

In [209]:
# Вычислите важности признаков и найдите два признака с наибольшей важностью.
# Их названия будут ответами для данной задачи
# (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

pd.Series(model.feature_importances_) # стоимость билета и пол оказались признаками с наибольшей важностью

0    0.140005
1    0.303436
2    0.256046
3    0.300512
dtype: float64